In [ ]:
# Boilerplate code from Deep Learning with Python to get started

In [3]:
from tensorflow.keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3


In [4]:
# Generator

generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16) (generator_input)
x = layers.LeakyReLU() (x)
x = layers.Reshape((16, 16, 128)) (x)

x = layers.Conv2D(256, 5, padding='same') (x)
x = layers.LeakyReLU() (x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same') (x)
x = layers.LeakyReLU() (x)

x = layers.Conv2D(256, 5, padding='same') (x)
x = layers.LeakyReLU() (x)
x = layers.Conv2D(256, 5, padding='same') (x)
x = layers.LeakyReLU() (x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same') (x)

generator = keras.models.Model(generator_input, x)

generator.summary()



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)      

In [6]:
# Discriminator

discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3) (discriminator_input)
x = layers.LeakyReLU() (x)
x = layers.Conv2D(128, 4, strides=2) (x)
x = layers.LeakyReLU() (x)
x = layers.Conv2D(128, 4, strides=2) (x)
x = layers.LeakyReLU() (x)
x = layers.Conv2D(128, 4, strides=2) (x)
x = layers.LeakyReLU() (x)
x = layers.Flatten() (x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid') (x)

discriminator = keras.models.Model(discriminator_input, x)

discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8
)

discriminator.compile(
    optimizer=discriminator_optimizer,
    loss='binary_crossentropy'
)

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 6, 6, 128)        

In [8]:
# GAN

discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')